# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [1]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import MatrixModifier
from medusa.medusa_lca import *
from medusa.timeline_builder import TimelineBuilder
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [2]:
from tests.databases import *
# db_abc_simple() # works
# db_abc_C_to_E() # works
# db_abc_C_and_B_to_E() # BREAKS THINGS
# db_abc_B_to_E() # BREAKS THINGS
# db_abc_B_to_E_simplified() # BREAKS THINGS
# db_abc_B_to_E_simplified_and_E_with_TD()
db_abc_loopA_with_biosphere()

100%|██████████| 1/1 [00:00<00:00, 5461.33it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 46603.38it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 48770.98it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 4/4 [00:00<00:00, 95325.09it/s]

Vacuuming database 


In [3]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [4]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 240.5555559538397


# `MEDUSA` LCA

In [5]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [6]:
import warnings
warnings.filterwarnings("ignore")

database_date_dict = {
            datetime.strptime("2008", "%Y"): 'background_2008',
            datetime.strptime("2024", "%Y"): 'background_2024',
        }

In [7]:
# tl_builder = TimelineBuilder(slca, filter_function, database_date_dict)

# timeline = tl_builder.build_timeline()

# demand_timing_dict = create_demand_timing_dict(timeline, demand) # Should go elsewhere as well - medusa LCA?

# mm = MatrixModifier(timeline, database_date_dict, demand_timing_dict)
# dp = mm.create_datapackage()

# fu, data_objs, remapping = prepare_medusa_lca_inputs(demand=demand, demand_timing_dict=demand_timing_dict, method=gwp) 
# lca = bc.LCA(fu, data_objs = data_objs + dp, remapping_dicts=remapping)
# lca.lci()
# lca.lcia()

In [8]:
mlca = MedusaLCA(slca, filter_function, database_date_dict)

Starting graph traversal
Calculation count: 42


In [9]:
mlca.build_timeline()

All databases in database_date_dict exist as brightway project databases


,hash_producer,date_producer,producer,producer_name,hash_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,1995,1995-01-01,2,C,2000,2000-01-01,8,B,13.0,{'background_2008': 1}
1,1996,1996-01-01,2,C,2001,2001-01-01,8,B,13.0,{'background_2008': 1}
2,1997,1997-01-01,2,C,2002,2002-01-01,8,B,13.0,{'background_2008': 1}
3,1998,1998-01-01,2,C,2003,2003-01-01,8,B,13.0,{'background_2008': 1}
4,1999,1999-01-01,2,C,2004,2004-01-01,8,B,13.0,{'background_2008': 1}
...,...,...,...,...,...,...,...,...,...,...
79,2035,2035-01-01,8,B,2020,2020-01-01,9,A,0.4,{'background_2024': 1}
80,2036,2036-01-01,8,B,2021,2021-01-01,9,A,0.4,{'background_2024': 1}
81,2037,2037-01-01,8,B,2022,2022-01-01,9,A,0.4,{'background_2024': 1}
82,2038,2038-01-01,8,B,2023,2023-01-01,9,A,0.4,{'background_2024': 1}


In [10]:
mlca.build_datapackage()

In [11]:
mlca.lci()

In [12]:
mlca.lcia()

In [13]:
print('New MEDUSA LCA Score:', mlca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: 240.3056850550947
Old static LCA Score: 240.5555559538397
